In [1]:
import pandas as pd
import os

In [53]:
pbCycles = {"pbCycle": ["0-", "0.05-", "0.1-", "0.15-", "0.2-", "0.25-", "0.3-", "0.35-", "0.4-",
                 "0.45-", "0.5-", "0.55-", "0.6-", "0.65-", "0.7-", "0.75-", "0.8-", "0.85-", "0.9-"]}
pbAtts = {"pbAtt": ["0.15-", "0.2-", "0.25-", "0.3-", "0.35-", "0.4-", "0.45-", "0.5-", "0.55-",
                    "0.6-", "0.65-", "0.7-", "0.75-", "0.8-", "0.85-", "0.9-", "0.95-"]}
featuresWS_10 = {"pbCycle": ["0.25-", "0.5-", "0.75-"], "beta": ["0-", "0.25-", "0.5-", "0.75-", "1-"], "baseDegree": ["2-","4-","6-","8-"]}
featuresWS_20 = {"pbCycle": ["0.25-", "0.5-", "0.75-"], "beta": ["0-", "0.25-", "0.5-", "0.75-", "1-"], "baseDegree": ["4-","8-","12-","16-"]}
featuresWS_100 = {"pbCycle": ["0.25-", "0.5-", "0.75-"], "beta": ["0-", "0.25-", "0.5-", "0.75-", "1-"], "baseDegree": ["20-","40-","60-","80-"]}

alphas = ["0_5", "1_0", "5_0", "10_0"]

features = {
    "BA": {
        "numArg": {
            "10": pbCycles,
            "20": pbCycles,
            "100": pbCycles,
        },
    },
    "ER": {
        "numArg": {
            "10": pbAtts,
            "20": pbAtts,
            "100": pbAtts,
        },
    },
    "WS": {
        "numArg": {
            "10": featuresWS_10,
            "20": featuresWS_20,
            "100": featuresWS_100,
        },
    },
}

In [54]:
for type_graph in features:
    dirname = f"results/{type_graph}"
    if os.path.isdir(dirname):
        for num_args in features[type_graph]:
            for num_arg in features[type_graph][num_args]:
                files = [f"{dirname}/{num_arg}/{file}" for file in os.listdir(f"{dirname}/{num_arg}")]
                for feature in features[type_graph][num_args][num_arg]:
                    values = features[type_graph][num_args][num_arg][feature]
                    df_res = pd.DataFrame(columns=[value.replace("-", "") for value in values],
                                          index=[alpha.replace("_", ".") for alpha in alphas])
                    for alpha in alphas:
                        for file in files:
                            if alpha in file:
                                df = pd.read_csv(file)
                                zeros_prop = {value.replace("-", ""): None for value in values}
                                for value in values:
                                    restricted_df = df[
                                        (df["filename"].str.contains(f"{num_args}{num_arg}")) &
                                        (df["filename"].str.contains(f"{feature}{value}"))
                                    ]
                                    zeros_prop[value.replace("-", "")] = restricted_df[restricted_df.distance==0].shape[0]/restricted_df.shape[0]
                                df_res.loc[alpha.replace("_", ".")] = pd.Series(zeros_prop)
                    df_res.to_csv(f"results/zeros_prop/{type_graph}_{num_arg}_{feature}.csv", index=True) 